In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Get Product Information from Search Page

In [9]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

# .get response code
r = requests.get('https://www.amazon.com/s?k=board+games&page=1', headers=headers)
content = r.content
soup = BeautifulSoup(content)

In [10]:
soup

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a

In [639]:
def get_data(no_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    for i in range(1, no_pages+1):
        r = requests.get('https://www.amazon.com/s?k=board+games&page='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)


        for d in soup.findAll('div', attrs={'class':'sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'}):
            name = d.find('span', attrs={'class':'a-size-base-plus a-color-base a-text-normal'})
            price = d.find('span', attrs={'class':'a-offscreen'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
            no_of_ratings = d.find('span', attrs={'class':'a-size-base'})
            asin = d['data-asin']

            all1=[]
            
            if asin is not None:
                all1.append(asin)
            else:
                all1.append('-')
            
            if name is not None:
                all1.append(name.text)
            else:
                all1.append("unknown-product")

            if price is not None:
                all1.append(price.text)
            else:
                all1.append('$0')

            if rating is not None:
                all1.append(rating.text)
            else:
                all1.append('-')
                
            if no_of_ratings is not None:
                all1.append(no_of_ratings.text)
            else:
                all1.append('-') 
                


            alls.append(all1)  

    return alls

In [640]:
df = pd.DataFrame(get_data(2),columns=['asin_id',
                                       'name',
                                       'price',
                                       'rating',
                                       'no_of_ratings'])

In [641]:
df.head(10)

,asin_id,name,price,rating,no_of_ratings
0,B076HK9H7Z,Sorry! Game,$0,4.7 out of 5 stars,"7,329"
1,B00D7OAOYI,Trouble Game,$0,4.5 out of 5 stars,"7,698"
2,B00C0ULS3G,Battleship Classic Board Game Strategy Game Ag...,$0,4.7 out of 5 stars,"3,976"
3,B08636S37Z,Funko Disney The Haunted Mansion – Call of The...,$24.97,4.4 out of 5 stars,15
4,B00000IVAK,SEQUENCE- Original SEQUENCE Game with Folding ...,$15.17,4.8 out of 5 stars,"11,519"
5,B07B7KS87Q,Game Of Life,$19.99,4.7 out of 5 stars,"4,159"
6,B08GD242TJ,Hasbro Gaming Guess Who? Game Original Guessin...,$9.97,4.8 out of 5 stars,14
7,B01JYVHMVA,Clue Game,$0,4.7 out of 5 stars,"5,206"
8,B00A2HD40E,Pandemic,$0,4.8 out of 5 stars,"7,422"
9,B088QSDB7S,Cephalofair Games Gloomhaven: Jaws of The Lion...,$41.50,4.9 out of 5 stars,366


In [653]:
df.to_csv('csv_files/amazon_items.csv')

## Get Links for each Item

In [642]:
# def get_links(df_column):
#     links = []
#     for product_id in df_column:
#         if product_id != 'Missing':
#             links.append('https://www.amazon.com/product-reviews/'+str(product_id)+'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')
#     return links

In [643]:
# product_links = get_links(df['Asin_No'])
# product_links[0:3]

In [644]:
# def content(link):
#     page=requests.get(link,headers=headers)
#     if page.status_code==200:
#         return page
#     else:
#         return "Error" 

# Get Reviews for each Item

In [647]:
def get_reviews(asin_no, no_review_pages):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    alls = []
    for i in range(1, no_review_pages+1):
        r = requests.get('https://www.amazon.com/product-reviews/'+asin_no+'/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber='+str(i), headers=headers)
        content = r.content
        soup = BeautifulSoup(content)

        for d in soup.findAll('div', attrs={'class':'a-section review aok-relative'}):
            product_id = d['id']
            review_title = d.find('a', attrs={'class':'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold'})
            review_description = d.find('span', attrs={'class':'a-size-base review-text review-text-content'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
#             user_name = d.find('span', attrs={'class': 'a-profile-name'}).text

            all1=[]
            
            all1.append(asin_no)
            
            if product_id is not None:
                all1.append(product_id)
            else:
                all1.append("unknown-product")

            if review_title is not None:
                all1.append(review_title.text)
            else:
                all1.append('$0')

            if review_description is not None:
                #print(rating.text)
                all1.append(review_description.text)
            else:
                all1.append('-1')

            if rating is not None:
                #print(price.text)
                all1.append(rating.text)
            else:
                all1.append('0')


            alls.append(all1)  

    return alls

In [648]:
get_reviews('B08636S37Z',2)

[['B08636S37Z',
  'R14GO0PH52D24X',
  '\nExtremely complicated and horrible instructions\n',
  "\n\n  A huge fan of the Haunted Mansion, I couldn't wait for this game to come out.  A huge, huge disappointment! I'm 57-yrs-old with an Ivy League law degree playing with my 16-year-old son and neither of us could figure out how to even play the game, even after reading the instructions over and over to try and make sense of them.  The instructions are horrible and totally vague and leave one even further confused and frustrated and they even provide a web address on Funko for an overview on how to play, but it takes you to a non-existent page. Funko blew this one big time and truly disappointing\n\n",
  '1.0 out of 5 stars'],
 ['B08636S37Z',
  'R8AP3Z0SIF2J6',
  '\nVery easy and replayable! Perfect for a Haunted Mansion fan!\n',
  '\n\n  I’m a huge Disney Parks fan, and have been looking forward to the release of a proper Haunted Mansion board game, especially since it was designed by the 

In [649]:
# append reviews from all items to list

reviews = []

for id_no in df['asin_id']:
    for row in get_reviews(id_no,5):
        reviews.append(row)

In [650]:
reviews

[['B076HK9H7Z',
  'R1OSPWS88F2CUZ',
  '\nDO NOT BUY!!!\n',
  '\n\n  I would give this zero stars if I could! If you’re into the original sorry game, DON’T buy this version. They changed the rules entirely while ruining the whole game! There are no longer 4 pawns per person, they only have 3! And rather than only being able to get out on a 1 or 2, you can now get out on any number.. oh and the “sorry!” card which allows you to bump someone back to start, now also has the option to move 4 places. It’s like this game was created for this new generation of entitled children that can’t take the time to finish a challenging game the right way without throwing a fit because they have instant gratification issues. Wish I opened it sooner so I could return it. I am very diasspointed in this update!!\n\n',
  '1.0 out of 5 stars'],
 ['B076HK9H7Z',
  'R1DCFJ8VYSN17B',
  '\nIs this the millennial version?\n',
  '\n\n  This is not the original sorry game. It only has 3 pawns per team instead of the 

In [651]:
df_reviews = pd.DataFrame(reviews,columns=['asin_id',
                                       'review_id',
                                       'review_title',
                                       'review_description',
                                       'rating'])

In [652]:
df_reviews

,asin_id,review_id,review_title,review_description,rating
0,B076HK9H7Z,R1OSPWS88F2CUZ,\nDO NOT BUY!!!\n,\n\n I would give this zero stars if I could!...,1.0 out of 5 stars
1,B076HK9H7Z,R1DCFJ8VYSN17B,\nIs this the millennial version?\n,\n\n This is not the original sorry game. It ...,1.0 out of 5 stars
2,B076HK9H7Z,R1V07N4GXA7RSL,\nWimp and Crybaby Edition\n,\n\n We bought this to replace our old Sorry ...,1.0 out of 5 stars
3,B076HK9H7Z,R2Z262NZDEU2EY,\nNOT the original/regular Sorry!\n,\n\n Be warned that this is not the sorry you...,2.0 out of 5 stars
4,B076HK9H7Z,RG3XIFV1PUX9Y,"\nNot the classic by a long shot, but okay.\n","\n\n Definitely not the classic game, with on...",4.0 out of 5 stars
...,...,...,...,...,...
4279,B07PXSJMHF,R2G37JKJ3LX0XX,\nGreat game\n,\n\n It's a really fun game. It's easy to le...,5.0 out of 5 stars
4280,B07PXSJMHF,R2H0T4QEAPAFI2,\nGreat family game.\n,\n\n Very well made and thick cards. Very eas...,5.0 out of 5 stars
4281,B07PXSJMHF,R1Y5NN7TY96BV3,\nThey captured minecraft in a board game.\n,\n\n You mine blocks to build or if you rathe...,5.0 out of 5 stars
4282,B07PXSJMHF,R2280MPKGGVKHF,\nGreat game\n,\n\n was wonder ful little games for children...,5.0 out of 5 stars


In [654]:
df_reviews.to_csv('csv_files/amazon_reviews.csv')